In [1]:
import pandas as pd

**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [43]:
df1 = df.groupby('userId').count()[['timestamp']].reset_index()
df1 = df1.rename(columns={'timestamp' : 'count_mark'})

In [44]:
df2 = df.groupby('userId').max()[['timestamp']].reset_index()
df2 = df2.rename(columns={'timestamp' : 'max_time'})

In [45]:
df3 = df.groupby('userId').min()[['timestamp']].reset_index()
df3 = df3.rename(columns={'timestamp' : 'min_time'})

In [47]:
df_res = df1.merge(df2, how='left').merge(df3, how='left')
len(df_res)

671

In [48]:
df_res_filter = df_res[df_res.count_mark>100]
len(df_res_filter)

258

In [52]:
df_res_filter['time_life'] = df_res_filter['max_time'] - df_res_filter['min_time']

C:\Users\Kosmo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
df_res_filter[['userId','count_mark','time_life']].head(10)

,userId,count_mark,time_life
3,4,204,203560
7,8,116,85187
14,15,1700,471393496
16,17,363,8053
18,19,423,5282
20,21,162,1365432
21,22,220,91491
22,23,726,18342129
25,26,172,20267261
29,30,1011,115852276


In [73]:
df_res_filter.time_life.mean()

40080507.4496124

**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента.
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [62]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [63]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [64]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [65]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [71]:
temp = (rzd
        .merge(auto, how='outer', on='client_id')
        .merge(air, how='outer', on='client_id')
        .fillna(0)
)
temp

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [72]:
temp2 = (rzd
        .merge(auto, how='outer', on='client_id')
        .merge(air, how='outer', on='client_id')
        .fillna(0)
        .merge(client_base, how='outer', on='client_id')
)
temp2

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Помимо широты и долготы хранить время в отдельном файле. У визита и покупкт так же есть время, таким образом можно будет связать по времени визит или факт покупки с координатами пользователя по известному Id.
2. Если у нас не будет признака, который связывает координаты и визит, то будет не понятно, как вообще связаны географическое координаты и данные о визите
3. Гегоафическая локация во время покупки, может пользователь увидел банер или что-то плдобное, т.е. например можно оценить эффективность наружной рекламы.